In [2]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#Import API key
from api_keys import g_key
#Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
#load csv into dataframe
load_csv = "../WeatherPy/output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Scarborough,54.28,-0.40,54.00,80,98,5.01,GB,1603123201
1,1,Mataura,-46.19,168.86,48.99,81,59,15.99,NZ,1603123426
2,2,Nome,64.50,-165.41,39.20,93,90,5.82,US,1603123426
3,3,Hirado,33.36,129.55,66.00,68,29,11.99,JP,1603123426
4,4,Tasiilaq,65.61,-37.64,32.00,97,40,8.05,GL,1603123426


In [6]:
#Humidity Heatmap

locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)


#Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,56,56,Mānsa,29.98,75.38,72.72,19,0,2.17,IN,1603123432
1,111,111,Marzuq,14.40,46.47,73.98,20,0,7.40,YE,1603123440
2,123,123,Guerrero Negro,27.98,-114.06,76.06,65,0,5.82,MX,1603123441
3,130,130,Rāmhormoz,31.28,49.60,72.82,19,0,3.06,IR,1603123442
4,188,188,Kamalpur,24.20,91.83,76.98,87,0,1.12,IN,1603123449
5,205,205,Najrān,17.49,44.13,77.00,22,0,6.93,SA,1603123451
6,254,254,Bannu,32.99,70.60,72.91,17,0,1.41,PK,1603123457
7,331,331,Vavuniya,8.75,80.50,77.31,83,0,6.44,LK,1603123466
8,342,342,Tall Kayf,37.08,41.07,78.80,23,0,6.93,TR,1603123467
9,428,428,Micheweni,-4.97,39.83,78.82,74,0,10.00,TZ,1603123478


In [8]:
# create hotel_df with hotel name column
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,56,Mānsa,29.98,75.38,72.72,19,0,2.17,IN,1603123432,NaN
1,111,Marzuq,14.40,46.47,73.98,20,0,7.40,YE,1603123440,NaN
2,123,Guerrero Negro,27.98,-114.06,76.06,65,0,5.82,MX,1603123441,NaN
3,130,Rāmhormoz,31.28,49.60,72.82,19,0,3.06,IR,1603123442,NaN
4,188,Kamalpur,24.20,91.83,76.98,87,0,1.12,IN,1603123449,NaN
5,205,Najrān,17.49,44.13,77.00,22,0,6.93,SA,1603123451,NaN
6,254,Bannu,32.99,70.60,72.91,17,0,1.41,PK,1603123457,NaN
7,331,Vavuniya,8.75,80.50,77.31,83,0,6.44,LK,1603123466,NaN
8,342,Tall Kayf,37.08,41.07,78.80,23,0,6.93,TR,1603123467,NaN
9,428,Micheweni,-4.97,39.83,78.82,74,0,10.00,TZ,1603123478,NaN


In [10]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
#checker variable
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,56,Mānsa,29.98,75.38,72.72,19,0,2.17,IN,1603123432,STARiHOTEL Mansa
1,111,Marzuq,14.40,46.47,73.98,20,0,7.40,YE,1603123440,NaN
2,123,Guerrero Negro,27.98,-114.06,76.06,65,0,5.82,MX,1603123441,"Casa Laguna, Bed & Breakfast"
3,130,Rāmhormoz,31.28,49.60,72.82,19,0,3.06,IR,1603123442,فروشگاه مطورعلیان
4,188,Kamalpur,24.20,91.83,76.98,87,0,1.12,IN,1603123449,Sagar Guest House. সাগর গেস্ট হাউস
5,205,Najrān,17.49,44.13,77.00,22,0,6.93,SA,1603123451,محمد هادي ال فطيح
6,254,Bannu,32.99,70.60,72.91,17,0,1.41,PK,1603123457,Aamir Hotel Bannu
7,331,Vavuniya,8.75,80.50,77.31,83,0,6.44,LK,1603123466,Thampa Hotel
8,342,Tall Kayf,37.08,41.07,78.80,23,0,6.93,TR,1603123467,NaN
9,428,Micheweni,-4.97,39.83,78.82,74,0,10.00,TZ,1603123478,NaN


In [11]:
#clean up DataSet by remooving NAN
hotel_df_clean = hotel_df.dropna()
hotel_df_clean

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,56,Mānsa,29.98,75.38,72.72,19,0,2.17,IN,1603123432,STARiHOTEL Mansa
2,123,Guerrero Negro,27.98,-114.06,76.06,65,0,5.82,MX,1603123441,"Casa Laguna, Bed & Breakfast"
3,130,Rāmhormoz,31.28,49.60,72.82,19,0,3.06,IR,1603123442,فروشگاه مطورعلیان
4,188,Kamalpur,24.20,91.83,76.98,87,0,1.12,IN,1603123449,Sagar Guest House. সাগর গেস্ট হাউস
5,205,Najrān,17.49,44.13,77.00,22,0,6.93,SA,1603123451,محمد هادي ال فطيح
6,254,Bannu,32.99,70.60,72.91,17,0,1.41,PK,1603123457,Aamir Hotel Bannu
7,331,Vavuniya,8.75,80.50,77.31,83,0,6.44,LK,1603123466,Thampa Hotel
10,430,Iranshahr,27.20,60.68,78.80,6,0,5.46,IR,1603123478,کافی نت دانشگاه


In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))